# Hadamard Pooling — Analysis Vector Checking Notebook

This notebook lets you:
1) Input **n**
2) Print full **S** matrix
3) Print type-2 **reduced S** matrix (drops last column)
4) Print **pool assignments** (A, B, …, Z, a, b, …)
5) Enter an **analysis vector** (e.g. 1,1,0,0 comma or 1 1 0 0 space separated) and identify **one or more positive samples** (sum of columns)

**Tip:** run all cells, then use the widget at the bottom.

In [ ]:
import numpy as np
import ipywidgets as wgt
from IPython.display import display
import re
from itertools import combinations

## Utility Functions

In [ ]:
def isprime(n: int) -> bool:
    if n < 2:
        return False
    if n % 2 == 0:
        return n == 2
    x = 3
    while x * x <= n:
        if n % x == 0:
            return False
        x += 2
    return True

def valid_seq_length(n: int) -> bool:
    """Quadratic residue possible when n is prime and n ≡ 3 (mod 4)."""
    return isprime(n) and (n % 4 == 3)

def row_labels(n: int):
    """Labels: A..Z, a..z, then aa, ab, ... if ever needed."""
    labels = []
    for i in range(n):
        if i < 26:
            labels.append(chr(65 + i))  # A-Z
        elif i < 52:
            labels.append(chr(97 + (i - 26)))  # a-z
        else:
            # double-lowercase (aa, ab, ...)
            x = i - 52
            parts = []
            while True:
                parts.append(chr(97 + (x % 26)))
                x = x // 26 - 1
                if x < 0:
                    break
            labels.append(''.join(reversed(parts)))
    return labels

## Matrix Constructions

In [ ]:
def quadratic_hadamard(n: int) -> np.ndarray:
    alist = np.zeros(n, dtype=int)
    for i in range(0, (n-1)//2):
        alist[(i+1)*(i+1) % n] = 1
    alist[0] = 1
    Srow = list(alist)
    S = np.zeros((n,n), dtype=int)
    for i in range(n):
        for j in range(n):
            S[i,j]= Srow[n-1-j]
        Srow = np.roll(Srow, 1)
    return S

def shift_hadamard(k: int) -> np.ndarray:
    n = 2**k - 1
    Srow = np.zeros(n, dtype=int)
    SL = np.zeros(k, dtype=int)
    SL[0] = 1
    for j in range(n):
        tmp = (SL[1] + SL[0]) % 2
        SL = np.roll(SL, -1)
        SL[-1] = tmp
        Srow[j] = SL[0]
    S = np.zeros((n,n), dtype=int)
    for i in range(n):
        for j in range(n):
            S[i,j] = Srow[n-1-j]
        Srow = np.roll(Srow, 1)
    return S

def doubling_hadamard(k: int) -> np.ndarray:
    H = np.array([[1, 1],[1, -1]])
    for _ in range(1, k):
        H = np.block([[H, H],[H, -H]])
    sm = H.shape[0] - 1
    Tmp = H[1:sm+1, 0:sm]
    Smat = np.where(Tmp == 1, 0, 1)
    return Smat

def legendre(p: int):
    leg = np.zeros(p, dtype=int)
    for i in range(p):
        temp = pow(i, (p-1)//2, p)
        if temp > 1:
            temp = -1
        leg[i] = temp
    return leg[1:]

def Hadamard28(L, p: int) -> np.ndarray:
    Had28 = np.zeros((2*(p+1), 2*(p+1)), dtype=int)
    B     = np.zeros((p+1, p+1), dtype=int)
    Q     = np.zeros((p, p), dtype=int)
    Q[0, 1:] = L[:]
    for i in range(1, p):
        Q[i, :] = np.roll(Q[0, :], i)
    Q[0, 0] = 0
    B[0, :] = 1
    B[:, 0] = 1
    for i in range(1, p+1):
        for j in range(1, p+1):
            B[i, j] = Q[i-1, j-1]
    B[0, 0] = 0
    for i in range(0, p+1):
        for j in range(0, p+1):
            if B[i, j] == 1:
                Had28[i+i-1, j+j-1] = 1
                Had28[i+i-1, j+j]   = 1
                Had28[i+i,   j+j-1] = 1
                Had28[i+i,   j+j]   = 0
            elif B[i, j] == -1:
                Had28[i+(i-1), j+(j-1)] = 0
                Had28[i+(i-1), j+j]     = 0
                Had28[i+i,     j+(j-1)] = 0
                Had28[i+i,     j+j]     = 1
            elif B[i, j] == 0:
                Had28[i+(i-1), j+(j-1)] = 1
                Had28[i+(i-1), j+j]     = 0
                Had28[i+i,     j+(j-1)] = 0
                Had28[i+i,     j+j]     = 0
    return Had28

## Matrix Generator (chooses largest valid size ≤ n)

In [ ]:
def generate_S(n: int, max_k: int = 12) -> np.ndarray:
    """Priority: Legendre (n=27) -> Quadratic Residue -> Shift Register -> Doubling.
    Always returns a square matrix with size ≤ n.
    """
    if n == 27:
        p = 13
        L = legendre(p)
        Had28 = Hadamard28(L, p)
        S27 = Had28[:-1, :-1]
        print("Generated 27×27 S-matrix using Legendre construction.")
        return S27

    candidates = []
    # Quadratic residue candidate ≤ n
    for m in range(n, 1, -1):
        if valid_seq_length(m):
            candidates.append(("Quadratic Residue", m))
            break
    # Shift register (2^k-1) ≤ n
    for kk in range(max_k, 1, -1):
        m = 2**kk - 1
        if m <= n:
            candidates.append((f"Shift Register (k={kk})", m))
            break
    # Doubling (2^k-1) ≤ n
    for kk in range(max_k, 1, -1):
        m = 2**kk - 1
        if m <= n:
            candidates.append((f"Doubling (k={kk})", m))
            break
    if not candidates:
        raise ValueError(f"No suitable Hadamard construction found for n={n}")
    method, size = max(candidates, key=lambda x: x[1])
    if "Quadratic Residue" in method:
        S = quadratic_hadamard(size)
    elif "Shift Register" in method:
        k = int(method.split("=")[-1].strip(")"))
        S = shift_hadamard(k)
    else:
        k = int(method.split("=")[-1].strip(")"))
        S = doubling_hadamard(k)
    print(f"Generated {S.shape[0]}×{S.shape[1]} S-matrix using {method}.")
    return S

## Row Selection (Type-2)

In [ ]:
def make_rows(m: int):
    """Manual rows for known sizes (Type-2)."""
    if m == 7:
        letters = ['A','D','F','G']
    elif m == 11:
        letters = ['A', 'C', 'G', 'H', 'I', 'K']
    elif m == 15:
        letters = ['A', 'B', 'C', 'D', 'E', 'F', 'I', 'J', 'K', 'O']
    elif m == 19:
        letters = ['A', 'C', 'D', 'I', 'K', 'M', 'N', 'O', 'P', 'S']
    elif m == 23:
        letters = ['A','F','H','I','K','O','P','R','T','V','W']
    elif m == 27:
        letters = ['A','B','C','D','E','F','G','H','J','M','O','T','X']
    elif m == 31:
        letters = ['C','D','G','J','L','N','P','U','V','X','Y','a','e']
    else:
        raise ValueError('make_rows not defined for this size')
    all_labs = row_labels(m)
    return [all_labs.index(l) for l in letters]

def _unique_columns(R: np.ndarray) -> bool:
    cols = [tuple(R[:, j]) for j in range(R.shape[1])]
    return len(set(cols)) == R.shape[1]

def _greedy_unique_rows(M: np.ndarray):
    """Greedy selection of rows to make all columns unique in M (fast for large m)."""
    n_rows, n_cols = M.shape
    selected = []
    sigs = [()] * n_cols  # current signatures per column
    while True:
        if len(set(sigs)) == n_cols:
            break
        best_row = None
        best_unique = -1
        for r in range(n_rows):
            if r in selected:
                continue
            new_sigs = [s + (int(M[r, j]),) for j, s in enumerate(sigs)]
            uniq = len(set(new_sigs))
            if uniq > best_unique:
                best_unique = uniq
                best_row = r
        if best_row is None:
            # fallback: include all rows
            return list(range(n_rows))
        selected.append(best_row)
        sigs = [s + (int(M[best_row, j]),) for j, s in enumerate(sigs)]
    return selected

def choose_rows(S: np.ndarray, n: int):
    """
    Use hard-coded rows when available; otherwise select rows automatically.
    Importantly, **uniqueness is enforced on the reduced matrix (drops last column)**,
    so we evaluate uniqueness on M = S[:, :-1].
    - For m ≤ 20, try exact minimal search.
    - For m > 20, use a greedy fast selector.
    Returns: rows, R (already reduced: M[rows, :])
    """
    m = S.shape[0]
    M = S[:, :-1]
    # Try manual rows
    try:
        rows = make_rows(m)
        R = M[rows, :]
        if _unique_columns(R):
            return rows, R
        else:
            print(f"⚠️ Warning: make_rows({m}) has duplicate columns after reduction; switching to automatic mode.")
    except Exception:
        pass
    # Automatic exact for small m
    if m <= 20:
        for k in range(1, m+1):
            for rows in combinations(range(m), k):
                R = M[list(rows), :]
                if _unique_columns(R):
                    return list(rows), R
    # Greedy for larger m
    rows = _greedy_unique_rows(M)
    R = M[rows, :]
    return rows, R

## Matching and Helpers

In [ ]:
def pool_assignments(R: np.ndarray):
    assigns = []
    for r in range(R.shape[0]):
        assigns.append([c+1 for c, v in enumerate(R[r, :]) if v == 1])
    return assigns

def column_matches(R: np.ndarray, v: np.ndarray) -> list[int]:
    return [j+1 for j in range(R.shape[1]) if np.array_equal(R[:, j], v)]

def sum_of_columns_matches(R: np.ndarray, v: np.ndarray, max_solutions: int = 20):
    R = np.array(R, dtype=int); v = np.array(v, dtype=int)
    cols = [j for j in range(R.shape[1]) if np.all(R[:, j] <= v)]
    cols.sort(key=lambda j: int(R[:, j].sum()), reverse=True)
    sols = []
    def dfs(start, residual, chosen):
        if len(sols) >= max_solutions:
            return
        if np.all(residual == 0):
            sols.append([c+1 for c in chosen]); return
        if np.any(residual < 0):
            return
        for idx in range(start, len(cols)):
            j = cols[idx]; col = R[:, j]
            if np.all(col <= residual):
                dfs(idx+1, residual - col, chosen + [j])
    dfs(0, v, [])
    return sols

def _fmt(v):
    return '[' + ', '.join(map(str, map(int, v))) + ']'

## Interactive Widget

In [ ]:
n_widget = wgt.IntText(value=7, description='n')
build_btn = wgt.Button(description='Build S matrix')
rv_widget = wgt.Text(
    value='',
    description='Analysis Vector',
    placeholder='e.g. 0,1,2,0 or 0 1 2 0'
)
check_btn = wgt.Button(description='Check')
out = wgt.Output()
state = {'S': None, 'R': None, 'labels': None}

def _build(_):
    with out:
        out.clear_output()
        n = int(n_widget.value)
        S = generate_S(n)
        print('Full S-matrix\n', S)
        rows, R = choose_rows(S, n)   # R is already reduced (n-1 columns)
        labels = [row_labels(S.shape[0])[r] for r in rows]
        print('\nReduced (rows', labels, ')\n', R)
        # clarify whether automatic
        try:
            manual_rows = make_rows(S.shape[0])
            if set(manual_rows) != set(rows):
                print("\nℹ️ Reduced matrix calculated automatically (no make_rows defined for this n).")
        except Exception:
            print("\nℹ️ Reduced matrix calculated automatically (no make_rows defined for this n).")
        print(f"\nTotal number of samples: {R.shape[1]}")
        print('\nPool assignments:')
        for lab, samp in zip(labels, pool_assignments(R)):
            print(f'  Pool {lab}: samples {samp}')
        state.update(S=S, R=R, labels=labels)

def _check(_):
    with out:
        if state['R'] is None:
            print('Build first'); return
        R = state['R']
        tokens = [t for t in re.split(r'[\s,]+', rv_widget.value.strip()) if t != '']
        try:
            v = np.array([int(t) for t in tokens], dtype=int)
        except:
            print('Use comma- or space-separated integers'); return
        if v.size != R.shape[0]:
            print(f'Length {v.size} must equal number of pools {R.shape[0]}'); return
        print('Results:', _fmt(v))
        # singles
        singles = column_matches(R, v)
        if singles:
            print('Exact single-sample match(es):', singles)
        # doubles
        doubles = []
        for i in range(R.shape[1]):
            for j in range(i+1, R.shape[1]):
                if np.array_equal(R[:, i] + R[:, j], v):
                    doubles.append((i+1, j+1))
        if doubles:
            print('Exact double-sample match(es):')
            for d in doubles:
                print('  •', d)
        # general multi-sample
        sols = sum_of_columns_matches(R, v)
        if sols:
            print('Positive Samples:')
            for s in sols:
                print('  •', s)
        if not singles and not doubles and not sols:
            print('No exact match')

build_btn.on_click(_build)
check_btn.on_click(_check)
display(n_widget, build_btn, rv_widget, check_btn, out)